In [1]:
num_sample = 5000
num_burn = 2000
sample_size = 800
n_cpu = 10

min_degree = 1
max_degree = 2

In [2]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1*min_degree,2,0.1,2,0.1,-2,0.1,2,0.1,0.1])

Y_chain, A_chain, L_chain = sample_network_chain(adj1, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 2


100%|██████████| 7000/7000 [00:18<00:00, 382.90it/s]


(0.6017424278172585,
 0.41514124999999996,
 0.53805,
 0.34363739382484965,
 0.36536884422186733)

In [3]:
res_truth1 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0
)
res_truth2 = agcEffect(
    adj1,
    tau, rho, nu, beta,
    treatment_allocation=0.3,
    R=100,
    burnin_R=100,
    seed=0
)

ground_truth1 = {}
ground_truth1['average'] = res_truth1['average']
ground_truth1['direct'] = res_truth1['direct_effect']
ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
ground_truth1['psi_zero'] = res_truth1['psi_zero']

ground_truth1

100%|██████████| 200/200 [00:00<00:00, 567.13it/s]


{'average': 0.6596249999999999,
 'direct': 0.3172,
 'indirect': 0.01352500000000001,
 'spillover_effect': 0.023562500000000042,
 'psi_1_gamma': 0.7563,
 'psi_0_gamma': 0.4391,
 'psi_zero': 0.41553749999999995}

In [4]:
from utils import run_pll, delete_all_files_in_folder
from run_pll import run_dr_em_raw

treat_p = 0.7
mispec = 'outcome'
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'mispec': mispec} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

delete_all_files_in_folder(f'run/run_dr_em_raw')

from utils import run_pll
from run_pll import run_dr_em_raw

treat_p = 0.3
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'mispec': mispec} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

delete_all_files_in_folder(f'run/run_dr_em_raw')

# read results
import numpy as np
from utils import *


results1 = np.load(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

Multiprocessing <function run_dr_em_raw at 0x124b0cdc0> in 1000 tasks, with 10 processes...
Multiprocessing finished.
Multiprocessing <function run_dr_em_raw at 0x124b0cdc0> in 1000 tasks, with 10 processes...
Multiprocessing finished.


100%|██████████| 1000/1000 [00:35<00:00, 28.41it/s]


{'columns': ['average',
  'direct',
  'indirect',
  'spillover_effect',
  'psi_1_gamma',
  'psi_0_gamma',
  'psi_zero'],
 'coverage_rate': array([0.888, 0.871, 0.934, 0.949, 0.822, 0.906, 0.899]),
 'bias': array([ 0.00871759,  0.02836992, -0.00681863, -0.0073431 ,  0.01571357,
        -0.01265636, -0.00531326]),
 'mse': array([0.00344561, 0.01041435, 0.00458686, 0.01228491, 0.0059519 ,
        0.00480724, 0.00817444]),
 'var': array([0.00336961, 0.0096095 , 0.00454037, 0.01223099, 0.00570499,
        0.00464706, 0.00814621]),
 'ci_length': array([0.21866086, 0.41270422, 0.25512268, 0.43274621, 0.28362896,
        0.2717543 , 0.35370916]),
 'true_effect': array([0.659625 , 0.3172   , 0.013525 , 0.0235625, 0.7563   , 0.4391   ,
        0.4155375])}

In [5]:
from tqdm import tqdm
from drnet import compute_avg_effects_std_from_raw

results1 = np.load(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)

var = []
var_sim = []
est = []
idx = 0
for i in tqdm(range(Y_chain.shape[0])):
    avg_effects, se_hac = compute_avg_effects_std_from_raw(results1[i,:,idx],adj1,h=2)
    var.append(se_hac**2*sample_size)
    var_sim.append(np.var(results1[i,:,idx]))
    est.append(avg_effects)

var_true = np.var(est)
print(np.mean(var), np.mean(var_sim), var_true*sample_size)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:04<00:00, 205.85it/s]

3.3022874017199437 3.2150750634161227 2.6956908813639773


In [ ]:
from utils import run_pll, delete_all_files_in_folder
from run_pll import run_dr_em_raw

treat_p = 0.7
mispec = None
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'mispec': mispec} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

delete_all_files_in_folder(f'run/run_dr_em_raw')

from utils import run_pll
from run_pll import run_dr_em_raw

treat_p = 0.3
args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj1,
         'i': i,
         'treatment_allocation': treat_p,
         'mispec': mispec} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr_em_raw, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}.npy', res_list_array_dr)

delete_all_files_in_folder(f'run/run_dr_em_raw')


# read results
import numpy as np
from utils import *


results1 = np.load(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}.npy', allow_pickle=True)
results2 = np.load(f'run/sim_results/sim_dr_em_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}.npy', allow_pickle=True)

compute_stats(results1, results2, ground_truth1, adj1)

Multiprocessing <function run_dr_em_raw at 0x124b0cdc0> in 1000 tasks, with 10 processes...


## Compare with AUTOG

In [ ]:
from run_pll import run_autognet_raw

mispec = 'outcome'

if mispec == 'outcome':
    np.random.seed(0)
    L_chain = np.random.binomial(n=1, p=0.2, size=L_chain.shape)

    treat_p = 0.7
    args = [{'Y_chain': Y_chain,
            'A_chain': A_chain,
            'L_chain': L_chain,
            'adj': adj1,
            'i': i,
            'treatment_allocation': treat_p} for i in range(len(Y_chain))]

    res_list_dr = run_pll(run_autognet_raw, args, processes=n_cpu)
    res_list_array_dr = np.array(res_list_dr)
    # save results
    np.save(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)
    delete_all_files_in_folder(f'run/run_autog_raw')


    treat_p = 0.3
    args = [{'Y_chain': Y_chain,
            'A_chain': A_chain,
            'L_chain': L_chain,
            'adj': adj1,
            'i': i,
            'treatment_allocation': treat_p} for i in range(len(Y_chain))]

    res_list_dr = run_pll(run_autognet_raw, args, processes=n_cpu)
    res_list_array_dr = np.array(res_list_dr)
    # save results
    np.save(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{treat_p}_{mispec}.npy', res_list_array_dr)
    delete_all_files_in_folder(f'run/run_autog_raw')


results1 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_{mispec}.npy')
results2 = np.load(f'run/sim_results/sim_ag_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_{mispec}.npy')

compute_stats_ag(results1, results2, ground_truth1)